In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns

import plotly.express as px
from Funtions_Cal import Coupon_rate_bond,cal_period_bond,Yield_to_marturity,market_prices,Modified_duration

In [2]:
TP_DN = pd.read_excel(r'D:\DATA\HOC_TAP\DO AN\DATA\TRAIPHIEUDOANHNGHIEP_TECH.xlsx')
TP_DN = TP_DN.rename(columns={'Thời gian ĐT (tháng)':'TG giữ đến đáo hạn'})

In [3]:
TP_DN['Kỳ hạn trái phiếu'] = cal_period_bond(TP_DN,'Ngày đáo hạn','Ngày phát hành')

d:\DATA\HOC_TAP\DO AN\XULI\Funtions_Cal.py:9: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data[col1] = pd.to_datetime(data[col1])


In [4]:
TP_DN['Lãi suất'] = TP_DN['Lãi suất'].map(lambda x : x*100)

In [5]:
TP_DN['Coupon_rate'] = TP_DN.apply(lambda x : Coupon_rate_bond(x['mệnh giá'],x['Lãi suất'],x['tần số trả lãi']),axis=1)

In [6]:
TP_DN['Market_price'] =TP_DN.apply(lambda x : market_prices(x['mệnh giá'],x['Lãi suất'],x['tần số trả lãi'],x['Kỳ hạn trái phiếu']),axis=1)

In [7]:
TP_DN['YTM'] = TP_DN.apply(lambda x : Yield_to_marturity(x['Market_price'],x['mệnh giá'],x['Kỳ hạn trái phiếu'],x['tần số trả lãi'],x['Coupon_rate']),axis=1)

In [8]:
TP_DN.columns

Index(['STT', 'Mã TP', 'Lãi suất', 'Tổ chức phát hành', 'TG giữ đến đáo hạn',
       'ĐT tối thiểu(tr)', 'Niêm yết', 'TS đảm bảo', 'Ưu đãi KH VIP',
       'Chào bán (tỷ)', 'Ngày phát hành', 'Ngày đáo hạn', 'mệnh giá',
       'tần số trả lãi', 'Mã TP.1', 'Kỳ hạn trái phiếu', 'Coupon_rate',
       'Market_price', 'YTM'],
      dtype='object')

In [9]:
TP_DN['Modified_duration']=TP_DN.apply(lambda x :Modified_duration(x['mệnh giá'],x['Lãi suất'],x['YTM'],x['Kỳ hạn trái phiếu'],x['tần số trả lãi']),axis=1 )

In [10]:
# Tổ chức phát hành 
# Công ty cố phần tập đoàn Massan
CTY_MASSAN = TP_DN[TP_DN['Tổ chức phát hành']=='Công Ty Cổ Phần Tập Đoàn Masan']
CTY_MASSAN.columns
# So ngay cua Cty 
'''Những thông tin cần xem:
    - lãi suất
    - Tần suất trả lãi
    - Marker_price
    - Modified_duration
    - và chart thể hiện tương quan so sánh của Modifiedduration với lãi 

'''
time_phathanh = CTY_MASSAN[['Ngày phát hành','Lãi suất']].rename(columns={'Ngày phát hành':'Date'})
time_kethuc = CTY_MASSAN[['Ngày đáo hạn','Lãi suất']].rename(columns={'Ngày đáo hạn':'Date'})

time_name = pd.concat([time_phathanh,time_kethuc],axis=0).drop_duplicates().reset_index()
time_name = time_name.sort_values(by =['Date'],ascending=False)
time_name = time_name.drop_duplicates(['Date'])

In [11]:
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

In [12]:
time_name['Date1'] = time_name['Date']

In [13]:
test = pd.pivot_table(data=time_name, 
                     index= 'Date', 
                     columns= 'Date1', 
                     values=  'Lãi suất', 
                     ).fillna(0)

In [14]:
px.imshow(test,x = test.columns, y = test.index)

In [15]:
TP_DN.columns

Index(['STT', 'Mã TP', 'Lãi suất', 'Tổ chức phát hành', 'TG giữ đến đáo hạn',
       'ĐT tối thiểu(tr)', 'Niêm yết', 'TS đảm bảo', 'Ưu đãi KH VIP',
       'Chào bán (tỷ)', 'Ngày phát hành', 'Ngày đáo hạn', 'mệnh giá',
       'tần số trả lãi', 'Mã TP.1', 'Kỳ hạn trái phiếu', 'Coupon_rate',
       'Market_price', 'YTM', 'Modified_duration'],
      dtype='object')

In [16]:
laisuat  = pd.DataFrame(TP_DN.groupby(['Lãi suất','Tổ chức phát hành'])['Lãi suất'].count()).rename(columns={'Lãi suất':'count'}).reset_index()

In [17]:
fig = px.bar(x = laisuat['Lãi suất'], y =laisuat['count'],color=laisuat['Tổ chức phát hành'])

fig.update_xaxes(type='category')
fig.update_layout(font=dict(size=10),template="plotly_white")
fig.update_xaxes(rangeselector_font_color='#000000',# rangeslider_visible=True,
                ticks= "outside",
                ticklabelmode= "period", 
                ticklen=12,
            
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1D", step="day", stepmode="backward"),
                dict(count=7, label="1W", step="day", stepmode="backward"),
                dict(count=1,label="1M", step="month", stepmode="backward"),
                dict(count=3,label="3M", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
        ])
    )  )
fig.update_yaxes(gridwidth=1,mirror="ticks",showgrid=True,title="Value")
fig.update_layout(  legend=dict(
            orientation="h",
            # entrywidth=70,
            yanchor="middle",
            y=-0.3,
            xanchor="center",
            x=0.5,
        ),margin=dict(l=20, r=20, t=10, b=20))
fig.update_traces(hovertemplate ='%{label:label}'+ '<br>%{text}',textposition = "none")
fig.update_layout(   
    paper_bgcolor='#F2F3F4',
    plot_bgcolor='#F2F3F4')
fig.update_layout(hovermode="x unified",font=dict(size=10))
fig.update_layout(
            xaxis=dict(
                tickformat = '%b %-d'   #<----- this is the format to get weekday
            ),
            
        
        )
fig.show()

In [18]:
tansuatloinhuan = TP_DN[['Lãi suất','YTM','Kỳ hạn trái phiếu','tần số trả lãi','Market_price','Modified_duration']]

In [19]:
tansuatloinhuan['Tong_ky_tra_lai'] = tansuatloinhuan['Kỳ hạn trái phiếu']*tansuatloinhuan['tần số trả lãi']
tansuatloinhuan['YTM'] = tansuatloinhuan['YTM'].map(lambda x : x*100)

C:\Users\Admin\AppData\Local\Temp\ipykernel_13280\3508305707.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Admin\AppData\Local\Temp\ipykernel_13280\3508305707.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [20]:
px.histogram(tansuatloinhuan,x=tansuatloinhuan['Lãi suất'],opacity=0.8,
                   log_y=True,)

In [21]:
tansuatloinhuan.columns

Index(['Lãi suất', 'YTM', 'Kỳ hạn trái phiếu', 'tần số trả lãi',
       'Market_price', 'Modified_duration', 'Tong_ky_tra_lai'],
      dtype='object')

In [22]:
YTM_price = tansuatloinhuan.groupby(['YTM'])['Market_price'].mean().reset_index()
YTM_price

,YTM,Market_price
0,10.814662,93649.3
1,11.102695,93520.5
2,11.130147,93508.3
3,12.147021,93067.8
4,14.660050,92065.5
...,...,...
67,156.358992,99134.1
68,157.090963,99260.0
69,161.143756,99561.1
70,165.327970,99739.3


In [23]:
fig = px.line(x = YTM_price['YTM'],y= YTM_price['Market_price'])
fig.update_xaxes(type='category')


In [24]:
price_rate = TP_DN[TP_DN['tần số trả lãi'] >=  2]
price_rate.columns
# price_rate = price_rate[['Lãi suất','Market_price']].sort_values(by=['Market_price'],ascending=False).drop_duplicates(['Market_price'])

Index(['STT', 'Mã TP', 'Lãi suất', 'Tổ chức phát hành', 'TG giữ đến đáo hạn',
       'ĐT tối thiểu(tr)', 'Niêm yết', 'TS đảm bảo', 'Ưu đãi KH VIP',
       'Chào bán (tỷ)', 'Ngày phát hành', 'Ngày đáo hạn', 'mệnh giá',
       'tần số trả lãi', 'Mã TP.1', 'Kỳ hạn trái phiếu', 'Coupon_rate',
       'Market_price', 'YTM', 'Modified_duration'],
      dtype='object')

In [25]:
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
X = price_rate.iloc[:, 2].values.reshape(-1, 1)  # iloc[:, 1] is the column of X
Y = price_rate.iloc[:, 17].values.reshape(-1, 1)  # df.iloc[:, 4] is the column of Y
linear_regressor = LinearRegression()
linear_regressor.fit(X, Y)
Y_pred = linear_regressor.predict(X)

# plt.scatter(X, Y)
# plt.plot(X, Y_pred, color='red')
# plt.show()


In [26]:
X = pd.DataFrame(X)
Y = pd.DataFrame(Y)
Y_pred = pd.DataFrame(Y_pred)

In [27]:
linear = pd.concat([X,Y,Y_pred],axis=1)
linear.columns = ['X','Y','Y_pred']

In [28]:
trace1 = px.scatter(x = linear['X'], y = linear['Y'])
trace1.add_trace(go.Scatter(x =linear['X'], y = linear['Y_pred']))

In [29]:
def Modified_duration(face_vl,coupon,Yield,Time,per_time):
  Data = pd.DataFrame()
  coupon_payment = [(coupon/per_time)/100*face_vl]*(Time*per_time-1)+[(coupon/per_time)/100*face_vl+face_vl]
  total_payment = Time*per_time
  Data['period']=pd.DataFrame(np.arange(1,total_payment+1),columns = ['Periods'])
  Data['Coupon_payment']= pd.DataFrame(coupon_payment)
  Data['D_CP']=Data['Coupon_payment'] / ((1+(Yield/per_time)/100) ** Data['period'])
  Data['PV/Total_DCP']= Data['D_CP']*Data['period'] / Data['D_CP'].sum()

  Duration = Data['PV/Total_DCP'].sum()/per_time
  Modified_Duration = Duration / (1+(Yield/per_time)/100)
  return Modified_Duration

Modified_duration(100,10,6,3,2)

2.605818794940178

In [30]:
# cho giá trị vào và lãi suất nếu tăng hoặc giảm thì mô hình nhưn thế nào

start_value = 98000

market_price = []

while start_value >= 0:
    market_price.append(start_value)
    start_value -= 2000
market_price = pd.DataFrame(market_price)
star_laisuat = 7.8
lai_suat = []
for i in range(30):
    lai_suat.append(star_laisuat)
    star_laisuat += 1
lai_suat = pd.DataFrame(lai_suat)



In [31]:
compare_modified_duration = pd.concat([market_price[:30],lai_suat],axis=1)
compare_modified_duration.columns = ['price','laisuat']

In [32]:
compare_modified_duration.columns


Index(['price', 'laisuat'], dtype='object')

In [33]:
px.line(x = compare_modified_duration['laisuat'], y = compare_modified_duration['price'])

In [57]:
import pandas as pd
import numpy as np
def price_change(rate,coupon_rate, frequency, face_value,settlement_date, maturity_date, change_in_year):
    try:
        settlement_date = pd.to_datetime(settlement_date, format="%d/%m/%Y")
    except:
        settlement_date = pd.to_datetime(settlement_date, format="%d-%m-%Y")

    try:
        maturity_date = pd.to_datetime(maturity_date, format="%d/%m/%Y")
    except:
        maturity_date = pd.to_datetime(maturity_date, format="%d-%m-%Y")

    data = pd.DataFrame()
    rate = rate/100
    coupon_rate = coupon_rate/100

    n =pd.to_numeric(((pd.to_datetime(maturity_date) - pd.to_datetime(settlement_date))/365).days)
    total_payment = n*frequency
    coupon_payment = coupon_rate/frequency * face_value
    payment = [coupon_payment]*(total_payment-1)+[coupon_payment+face_value]
    data['period'] = pd.DataFrame(np.arange(1,total_payment+1))
    data['payment'] = pd.DataFrame(payment)
    data['dcoupon'] = data['payment']/((1+ rate/frequency)**data['period'])
    data['pv'] = data['dcoupon'] / frequency * data['period'] / data['dcoupon'].sum()
    duration = data['pv'].sum()
    m_duration = (duration/(1+rate/frequency))

    # convexity calculation 

    factor = 1/ (data['dcoupon'].sum()*(1+rate/frequency)**2)
    data['cf'] = data['dcoupon']*(data['period']**2 + data['period'])/ (1+rate/frequency)**data['period']
    convexity = factor * data['cf'].sum()

    # price change by using duration and converxity
    # price change = Duration Effect + converxity
    price_change = (-duration*change_in_year*100)+(0.5*convexity*(change_in_year)**2*100)


    return duration, m_duration, convexity,price_change,data